In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split

# Import custom modules
from network import *
from data import *

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Create the base network
network = dict(n_dense=10, dense_units=16, activation='selu', dropout=AlphaDropout, dropout_rate=0.1,
               kernel_initializer='lecun_normal', optimizer='sgd', num_classes=2)

shared_model = create_base_network(**network)

print("Shared model summary")
shared_model.summary()

Shared model summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                15664     
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
alpha_dropout_2 (AlphaDropou (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                

In [3]:
# Create the siamese network
model = create_siamese_network(shared_model)
print("Siamese network model summary")
model.summary()

Siamese network model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           18112       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
man_dist_1 (ManDist)            (None, 1)            0           sequential_1[1

In [4]:
# Obtain data
gctxfile = "../../Data/Sig Annotated Level 5 Data.gctx"
jsonfile = "../../Data/sig-pert mapping.json"

data, metadata = gctx2pd(gctxfile, jsonfile)
data.head()

cid,REP.A001_A375_24H:A03,REP.A001_A375_24H:A04,REP.A001_A375_24H:A05,REP.A001_A375_24H:A06,REP.A001_A375_24H:A07,REP.A001_A375_24H:A08,REP.A001_A375_24H:A09,REP.A001_A375_24H:A10,REP.A001_A375_24H:A11,REP.A001_A375_24H:A12,...,LJP007_SKL_24H:P19,LJP007_SKL_24H:P20,LJP007_SKL_24H:P21,LJP007_SKL_24H:P22,LJP007_SKL_24H:E21,LJP007_SKL_24H:O13,LJP007_SKL_24H:O14,LJP007_SKL_24H:O24,LJP007_SKL_24H:P24,LJP007_SKL_24H:C19
rid,,,,,,,,,,,,,,,,,,,,,
780,4.264143,-0.382211,-0.571711,0.584376,0.658348,-0.004232,-0.314762,-0.049558,-0.909517,-0.850654,...,1.091158,0.264409,0.711080,0.768569,4.4460,4.4395,6.1750,8.0582,10.0000,3.0807
7849,0.057249,0.304313,-0.754999,-0.589973,-0.226854,-0.363419,-0.691129,-0.684283,0.521503,-0.640316,...,-0.493212,-0.041785,-0.606896,0.819984,6.6313,10.0000,2.8649,0.4905,9.1524,4.5834
6193,-2.139334,-0.995924,-0.710110,-0.026398,-1.143599,-0.850314,-1.052307,-0.463051,-0.494277,0.067007,...,0.807524,-0.062587,0.632009,1.067584,3.4302,1.6831,1.8397,3.3238,-1.2545,0.3805
23,-0.221784,-0.670834,0.428894,-0.065268,0.342426,0.539448,0.357474,-0.233215,-0.715200,-0.418760,...,0.255770,-0.218802,0.257857,0.170097,2.6837,4.5242,4.3375,3.6885,-4.9315,2.2650
9552,-0.376555,-0.648242,0.272606,0.542223,0.380470,-0.011210,-1.590959,-0.613891,-0.291982,-0.629392,...,0.301324,-0.841693,0.628758,-0.387287,-1.6202,-2.6985,-1.0555,-1.5548,0.9744,-1.9243


In [5]:
# Obtain targets
target = get_target_labels(data, metadata)
print(len(target))

Creating target labels
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000 90000 91000 92000 93000 94000 95000 96000 97000 98000 99000 100000 101000 102000 103000 104000 105000 106000 107000 108000 109000 110000 111000 112000 113000 114000 115000 116000 117000 118000 118050


In [6]:
# Attach labels
data1 = data.transpose()
data1['target'] = target
data = data1.sort_values('target')
data.head()

rid,780,7849,6193,23,9552,387,10921,10285,533,6194,...,11000,6915,6253,7264,5467,2767,23038,57048,79716,target
cid,,,,,,,,,,,,,,,,,,,,,
REP.A012_HA1E_24H:B12,0.390700,-0.348900,0.617500,-1.761550,0.235550,1.574200,-0.337250,0.296200,0.535400,-0.937500,...,5.095200,-1.075550,-1.001950,0.247400,-1.916200,1.100850,-0.608800,1.202550,-0.400500,BRD-A00147595
REP.A012_HA1E_24H:B10,-0.354954,0.284711,-0.399639,-0.394218,0.269330,0.438478,-0.193475,-0.567282,0.599511,-0.201206,...,0.470325,0.660018,-0.167005,0.138563,0.316941,-0.231024,-0.122175,0.483970,0.055153,BRD-A00147595
REP.A012_HA1E_24H:B11,0.186348,-0.552110,-1.293667,1.087279,0.009011,-0.009710,-0.422903,0.122181,0.304543,0.021282,...,1.642745,2.120592,1.446014,-0.730442,-0.175147,-1.005282,0.414229,0.341628,1.157403,BRD-A00147595
REP.A012_MCF7_24H:B12,-0.409912,-0.012908,-0.357194,0.637424,-0.161387,0.674010,-0.540815,0.313900,-0.459992,0.065468,...,-0.730799,0.342100,-0.226505,0.312267,0.219967,0.585921,0.435886,0.102224,0.463525,BRD-A00147595
REP.A012_MCF7_24H:B08,-0.456875,-0.164637,-0.705794,-0.402885,-2.056623,-0.542763,0.000000,-1.418116,-0.543096,0.506668,...,-0.664681,-0.162204,-2.875566,0.113363,-0.143262,0.889934,0.909341,0.317749,0.296469,BRD-A00147595


In [7]:
# Create the 2 dictionaries
location_pert = create_location_pert(data)

creating location_pert
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 Creating pert2profile


In [9]:
pert2profiles = create_pert2profile(data)

Creating pert2profile


In [23]:
# Create pairs and targets
X, y = get_training_data(data, pert2profiles, location_pert, 1000)
print(X.shape)

2170
(2, 1000, 978)


In [25]:
# Fit data
model.fit([X[0], X[1]], y, epochs=100, verbose=1)

Epoch 1/100
1000/1000 [==============================] - 0s 223us/step - loss: 0.2818 - acc: 0.4800
Epoch 2/100
1000/1000 [==============================] - 0s 197us/step - loss: 0.2955 - acc: 0.5030
Epoch 3/100
1000/1000 [==============================] - 0s 227us/step - loss: 0.2809 - acc: 0.4780
Epoch 4/100
1000/1000 [==============================] - 0s 224us/step - loss: 0.2893 - acc: 0.4870
Epoch 5/100
1000/1000 [==============================] - 0s 215us/step - loss: 0.2902 - acc: 0.4950
Epoch 6/100
1000/1000 [==============================] - 0s 201us/step - loss: 0.2858 - acc: 0.5140
Epoch 7/100
1000/1000 [==============================] - 0s 194us/step - loss: 0.2923 - acc: 0.5030
Epoch 8/100
1000/1000 [==============================] - 0s 203us/step - loss: 0.2828 - acc: 0.4840
Epoch 9/100
1000/1000 [==============================] - 0s 192us/step - loss: 0.3031 - acc: 0.5300
Epoch 10/100
1000/1000 [==============================] - 0s 213us/step - loss: 0.2979 - acc: 0.4960

1000/1000 [==============================] - 0s 187us/step - loss: 0.2754 - acc: 0.4990
Epoch 83/100
1000/1000 [==============================] - 0s 327us/step - loss: 0.2821 - acc: 0.5040
Epoch 84/100
1000/1000 [==============================] - 0s 207us/step - loss: 0.2785 - acc: 0.5110
Epoch 85/100
1000/1000 [==============================] - 0s 221us/step - loss: 0.2910 - acc: 0.5200
Epoch 86/100
1000/1000 [==============================] - 0s 212us/step - loss: 0.2818 - acc: 0.5020
Epoch 87/100
1000/1000 [==============================] - 0s 295us/step - loss: 0.2835 - acc: 0.5010
Epoch 88/100
1000/1000 [==============================] - 0s 314us/step - loss: 0.2666 - acc: 0.4810
Epoch 89/100
1000/1000 [==============================] - 0s 374us/step - loss: 0.2787 - acc: 0.4980
Epoch 90/100
1000/1000 [==============================] - 0s 195us/step - loss: 5.7808 - acc: 0.4530
Epoch 91/100
1000/1000 [==============================] - 0s 197us/step - loss: 0.5005 - acc: 0.4870
Epo

In [106]:
X,y=generate_data(data,test,1)

BRD-A00218260 0 2
BRD-A00938334 0 4
